In [51]:
import os
import json
import pandas as pd
import traceback

In [52]:
from dotenv import load_dotenv
load_dotenv()

True

In [53]:
key=load_dotenv('OPENAI_API_KEY')

In [54]:
from langchain.chat_models import ChatOpenAI
client=ChatOpenAI(openai_api_key=key,model_name='gpt-4o',temperature=0.9)

In [55]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [56]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [57]:
TEMPLATE1="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [58]:
quiz_generation_prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE1
)

In [59]:
chain1=LLMChain(llm=client,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [60]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [61]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=['quiz','subject'],
    template=TEMPLATE2
)

In [62]:
chain2=LLMChain(llm=client,prompt=quiz_evaluation_prompt,output_key='review',verbose=True)

In [63]:
FinalChain=SequentialChain(chains=[chain1,chain2],input_variables=['text','number','subject','tone','response_json'],output_variables=['review'],verbose=True)

In [64]:
filepath=r'E:\GENAI\MCQ-GEN\paragraph.txt'

In [65]:
filepath

'E:\\GENAI\\MCQ-GEN\\paragraph.txt'

In [66]:
with open(filepath,'r') as file:
    data=file.read()

In [67]:
data

"Coronavirus disease 2019 (COVID-19) is a contagious disease caused by the coronavirus SARS-CoV-2. The first known case was identified in Wuhan, China, in December 2019.[7] Most scientists believe the SARS-CoV-2 virus entered into human populations through natural zoonosis, similar to the SARS-CoV-1 and MERS-CoV outbreaks, and consistent with other pandemics in human history.[8][9] Social and environmental factors including climate change, natural ecosystem destruction and wildlife trade increased the likelihood of such zoonotic spillover.[10][11][12][13] The disease quickly spread worldwide, resulting in the COVID-19 pandemic.\n\nThe symptoms of COVIDâ€‘19 are variable but often include fever,[14] fatigue, cough, breathing difficulties, loss of smell, and loss of taste.[15][16][17] Symptoms may begin one to fourteen days after exposure to the virus. At least a third of people who are infected do not develop noticeable symptoms.[18][19] Of those who develop symptoms noticeable enough t

In [68]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [69]:
NUMBER=5 
SUBJECT="covid"
TONE="simple"

In [70]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=FinalChain(
        {
            "text": data,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Coronavirus disease 2019 (COVID-19) is a contagious disease caused by the coronavirus SARS-CoV-2. The first known case was identified in Wuhan, China, in December 2019.[7] Most scientists believe the SARS-CoV-2 virus entered into human populations through natural zoonosis, similar to the SARS-CoV-1 and MERS-CoV outbreaks, and consistent with other pandemics in human history.[8][9] Social and environmental factors including climate change, natural ecosystem destruction and wildlife trade increased the likelihood of such zoonotic spillover.[10][11][12][13] The disease quickly spread worldwide, resulting in the COVID-19 pandemic.

The symptoms of COVIDâ€‘19 are variable but often include fever,[14] fatigue, cough, breathing difficulties, loss of smell, and loss of taste.[15][16][17] Symptoms may begin one to fourteen days after exposure to the virus. At least a third of people who a

In [71]:
print(f"TotalCost:{cb.total_cost}")
print(f"TotalToken:{cb.total_tokens}")
print(f"InputTokens{cb.prompt_tokens}")
print(f"OutputTokens{cb.completion_tokens}")

TotalCost:0.0178
TotalToken:2120
InputTokens1400
OutputTokens720


In [72]:
response

{'text': "Coronavirus disease 2019 (COVID-19) is a contagious disease caused by the coronavirus SARS-CoV-2. The first known case was identified in Wuhan, China, in December 2019.[7] Most scientists believe the SARS-CoV-2 virus entered into human populations through natural zoonosis, similar to the SARS-CoV-1 and MERS-CoV outbreaks, and consistent with other pandemics in human history.[8][9] Social and environmental factors including climate change, natural ecosystem destruction and wildlife trade increased the likelihood of such zoonotic spillover.[10][11][12][13] The disease quickly spread worldwide, resulting in the COVID-19 pandemic.\n\nThe symptoms of COVIDâ€‘19 are variable but often include fever,[14] fatigue, cough, breathing difficulties, loss of smell, and loss of taste.[15][16][17] Symptoms may begin one to fourteen days after exposure to the virus. At least a third of people who are infected do not develop noticeable symptoms.[18][19] Of those who develop symptoms noticeable

In [73]:
res=response.get("review")

In [74]:
print(res)

### Complexity Analysis

The quiz is reasonably challenging but accessible. The questions vary in complexity, requiring students to recall specific facts, understand general knowledge about COVID-19, and differentiate between related concepts. This balance ensures engagement without overwhelming.

### Suggested Updates

1. **Current Question 4:**
   - **Original:** "How long can people remain contagious after contracting COVID-19?"
   - **Complexity Issue:** The time frame options may be too specific and complex for some students.
   - **Updated:** "For how many days can people be contagious after getting COVID-19?"
     - **Options:** 
       - a) Up to 5 days
       - b) Up to 10 days
       - c) Up to 15 days
       - d) Up to 20 days
     - **Correct:** d

2. **Current Question 5:**
   - **Original:** "Which method is NOT used for COVID-19 testing?"
   - **Complexity Issue:** The terminology might be too technical.
   - **Updated:** "Which of these is NOT a method for testing COVID

In [75]:

# Extract the JSON part from the response
start = res.find("```json") + len("```json")
end = res.find("```", start)
json_str = res[start:end].strip()

# Print the extracted JSON string
print("Extracted JSON string:", json_str)

# Parse the JSON string
try:
    quiz = json.loads(json_str)
    print("Parsed JSON:", quiz)
except json.JSONDecodeError as e:
    print("JSONDecodeError:", e)
    print("JSON string that caused the error:", json_str)


Extracted JSON string: mplexity Analysis

The quiz is reasonably challenging but accessible. The questions vary in complexity, requiring students to recall specific facts, understand general knowledge about COVID-19, and differentiate between related concepts. This balance ensures engagement without overwhelming.

### Suggested Updates

1. **Current Question 4:**
   - **Original:** "How long can people remain contagious after contracting COVID-19?"
   - **Complexity Issue:** The time frame options may be too specific and complex for some students.
   - **Updated:** "For how many days can people be contagious after getting COVID-19?"
     - **Options:** 
       - a) Up to 5 days
       - b) Up to 10 days
       - c) Up to 15 days
       - d) Up to 20 days
     - **Correct:** d

2. **Current Question 5:**
   - **Original:** "Which method is NOT used for COVID-19 testing?"
   - **Complexity Issue:** The terminology might be too technical.
   - **Updated:** "Which of these is NOT a method 

In [76]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [77]:
quiz_table_data

[]

In [78]:
quiz=pd.DataFrame(quiz_table_data)


In [79]:
quiz

""


In [80]:
quiz.to_csv("Covid19.csv",index=False)